In [9]:
from openai import OpenAI
import json
from dotenv import load_dotenv

load_dotenv()

client = OpenAI()

# 1. Define a list of callable tools for the model
tools = [
    {
        "type": "function",
        "name": "get_movie_poster",
        "description": "Get poster of a given movie.",
        "parameters": {
            "type": "object",
            "properties": {
                "movie": {
                    "type": "string",
                    "description": "name of movie",
                },
            },
            "required": ["movie"],
        },
    },
]

def get_movie_poster(movie):
    return 'https://m.media-amazon.com/images/I/61sZGNr75iL._SL1000_.jpg'

# Create a running input list we will add to over time
input_list = [
    {"role": "user", "content": "what do you see in the poster of 3idiots"}
]

# 2. Prompt the model with tools defined
response = client.responses.create(
    model="gpt-4o-mini",
    tools=tools,
    input=input_list,
)

# Save function call outputs for subsequent requests
input_list += response.output

for item in response.output:
    if item.type == "function_call":
        if item.name == "get_movie_poster":
            # 3. Execute the function logic for get
            tool_output = get_movie_poster(json.loads(item.arguments))
            
            # 4. Provide function call results to the model
            input_list.append({
                "type": "function_call_output",
                "call_id": item.call_id,
                "output": {
                  "type": 'input_image',
                  "image_url": tool_output
                }
            })

print("Final input:")
print(input_list)

response = client.responses.create(
    model="gpt-4o-mini",
    instructions="Respond only with the description of the image by using the tool output",
    tools=tools,
    input=input_list,
)

# 5. The model should be able to give a response!
print("Final output:")
# print(response.model_dump_json(indent=2))
print("\n" + response.output_text)

Final input:
[{'role': 'user', 'content': 'what do you see in the poster of 3idiots'}, ResponseFunctionToolCall(arguments='{"movie":"3 Idiots"}', call_id='call_2OcyQ5030PIuBWVNebQMj1DK', name='get_movie_poster', type='function_call', id='fc_0193791851f27c8e00693b22a67b2c81909a696727ba4ce746', status='completed'), {'type': 'function_call_output', 'call_id': 'call_2OcyQ5030PIuBWVNebQMj1DK', 'output': {'type': 'input_image', 'image_url': 'https://m.media-amazon.com/images/I/61sZGNr75iL._SL1000_.jpg'}}]


BadRequestError: Error code: 400 - {'error': {'message': "Invalid type for 'input[2].output': expected one of a string or array of objects, but got an object instead.", 'type': 'invalid_request_error', 'param': 'input[2].output', 'code': 'invalid_type'}}